In [907]:
%matplotlib inline  
import numpy as np 
import scipy as sp 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import norm  
sns.set_style('white') 
sns.set_context('talk')  
np.random.seed(123)

import numpy as np

import scipy.special as sp
from scipy import integrate

from sklearn import metrics
from sklearn.cluster import DBSCAN

from sklearn.preprocessing import scale


In [ ]:
# def CrosValidEM(X_train, y_train, cv = 10):
#     Data_train = []
#     Data_valid = []
#     Answ_train = []
#     Answ_valid = []
    
#     Return = []
    
#     for i in range(cv):
#         x_t,  x_v, y_t, y_v = train_test_split(X_train, y_train_load, test_size = 1.0/cv)
#         Data_train.append(x_t)
#         Data_valid.append(x_v)
#         Answ_train.append(y_t)
#         Answ_valid.append(y_v)
    
#     for x_t,  x_v, y_t, y_v in zip(Data_train, Data_valid, Answ_train, Answ_valid):
#         M = np.array([np.mean(x_t[np.where(y_t == i)[0]], axis = 0) for i in range(2)])
#         D = np.array([np.std(x_t[np.where(y_t == i)[0]], axis = 0) for i in range(2)])

#         w, mu, cov = em.EM_algorithm(X_train=x_t,
#                                      X_test=X_test,
#                                      y_train=y_t,
#                                      count_of_clasters=2,
#                                      w=None,
#                                      mu=M,
#                                      cov=None,
#                                      alpha=0.0,
#                                      Num=100
#                                     )
        
#         Return.append([y_v, prob_class(x_v, mu, cov, w, count_of_labels = 2)[1]])
#     return Return
    
    
# a = CrosValidEM(X_train, y_train, cv = 10)
# ACC = []
# AUC = []
# NUM = []
# ASY1 = []
# ASY2 = []


# for b in a:
#     ASY1.append(_asy1(b[0], np.int64((np.sign(b[1] - 0.5) + 1)/2)))
#     ASY2.append(_asy2(b[0], np.int64((np.sign(b[1] - 0.5) + 1)/2)))
#     AUC.append(metrics.roc_auc_score(b[0], b[1]))
#     NUM.append(_num(b[0], np.int64((np.sign(b[1] - 0.5) + 1)/2)))
#     ACC.append(_acc(b[0], np.int64((np.sign(b[1] - 0.5) + 1)/2)))
    
    
# print('ACC:', np.mean(ACC), np.std(ACC))
# print('AUC:', np.mean(AUC), np.std(AUC))
# print('NUM:', np.mean(NUM), np.std(NUM))
# print('ASY1:', np.mean(ASY1), np.std(ASY1))
# print('ASY2:', np.mean(ASY2), np.std(ASY2))


# ACC = cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=X_train, y=y_train, scoring='roc_auc', cv = 10) 
# AUC = cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=X_train, y=y_train, scoring='accuracy', cv = 10)
# NUM = cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=X_train, y=y_train, scoring=num, cv = 10)
# ASY1 = cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=X_train, y=y_train, scoring=asy1, cv = 10)
# ASY2 = cross_val_score(estimator=svm.SVC(probability=True, kernel='rbf'), X=X_train, y=y_train, scoring=asy2, cv = 10)

# print('ACC:', np.mean(ACC), np.std(ACC))
# print('AUC:', np.mean(AUC), np.std(AUC))
# print('NUM:', np.mean(AUC), np.std(AUC))
# print('ASY1:', np.mean(ASY1), np.std(ASY1))
# print('ASY2:', np.mean(ASY2), np.std(ASY2))


In [889]:
def print_err(y, answ):
    Penalty_1 = np.array([[-10,10], [1,0]])
    Penalty_2 = np.array([[-1,2], [1,-1]])
    
    fpr, tpr, thresholds = metrics.roc_curve(y+1, answ+1, pos_label=2)
    print('AUC', metrics.auc(fpr, tpr))

    print('NUM', np.sum(answ != y))

    print('ASY1:', np.sum(Penalty_1[y, answ]))
    print('ASY2:', np.sum(Penalty_2[y, answ]))

    print('ACC:', np.sum(answ == y)/np.size(y))
    
def print_err_from_list(error):
    print('AUC', error[0])

    print('NUM', error[1])

    print('ASY1:', error[2])
    print('ASY2:', error[3])

    print('ACC:', error[4])
    
def get_err(y, answ):
    Penalty_1 = np.array([[-10,10], [1,0]])
    Penalty_2 = np.array([[-1,2], [1,-1]])
    
    fpr, tpr, thresholds = metrics.roc_curve(y+1, answ+1, pos_label=2)
    
    return metrics.auc(fpr, tpr), np.sum(answ != y), np.sum(Penalty_1[y, answ]), np.sum(Penalty_2[y, answ]), np.sum(answ == y)/np.size(y)

In [732]:
def _p_norm(x, mu = np.zeros(shape = 1), var = np.eye(1, dtype=np.float)):
    x = np.array(x)
    mu = np.array(mu)
    var = np.array(var)
    n = np.size(mu)
    m = x.shape[0]
    res = []
    for i in range(m):
        coef = (1.0/np.sqrt((((2*np.pi)**n)*np.linalg.det(var))))
        xi = x[i]
        exp = np.exp(-0.5*np.matmul(np.reshape(xi-mu, [1,-1]), np.matmul(np.linalg.inv(var), np.reshape(xi-mu, [-1,1]))))[0][0]
        res.append(coef*exp)
    return np.array(res)

def _p_norm_1d(x, mu = 0, var = 1.0):
    return 1.0/(np.sqrt(2*np.pi*var))*np.exp(-((x - mu)**2)/(2*var))


In [1105]:
def EM_algorithm(X, y_train, count_of_clasters = 2, mu = np.ones(shape=[2, 20]), Num = 50):
    count_of_data = X.shape[0]
    count_of_features = X.shape[1]
#     mu = np.ones(shape=[count_of_clasters, count_of_features])
#     mu = np.reshape(np.random.rand(count_of_clasters*count_of_features)- 0.5, [count_of_clasters, count_of_features])
    w = np.ones(shape=[count_of_clasters])/count_of_clasters
    cov = []
    for i in range(count_of_clasters):
        cov.append(np.eye(X.shape[1]))
    cov = np.array(cov)
    
    g = np.zeros(shape=[count_of_clasters, count_of_data])
    
    for i in range(Num):
# E-step
        Sum = 0
        for j in range(count_of_clasters):
            p = _p_norm(X, mu[j], cov[j])
            p[np.where(y_train==j)[0]] = 1
            Sum += w[j]*p
        for j in range(count_of_clasters):
            p = _p_norm(X, mu[j], cov[j])
            p[np.where(y_train==j)[0]] = 1
            g[j] = w[j]*p
#             g[j][np.where(y_train==j)[0]] = 1
            
# M-step
        N = np.sum(g, axis=1)
        for j in range(count_of_clasters):
            mu[j] = 1.0/N[j] * np.sum(np.multiply(g[j], X.T), axis =1)
            covariance = np.zeros_like(cov[j])
            for t in range(count_of_data):
                covariance += g[j][t] * np.matmul(np.reshape(X[t] - mu[j], [-1,1]), np.reshape(X[t] - mu[j], [1,-1]))
            cov[j] = 1.0/N[j] * covariance
        
        w = N/count_of_data
        
    return mu, cov

In [482]:
def Belsly(X):
    CovarianceMatrix = np.cov(X.T)
    U, S, V =  np.linalg.svd(CovarianceMatrix)
    S = np.sqrt(S)
    
    Q = np.zeros(shape = U.shape)
    n = Q.shape[0]
    for i in range(n):
        Sum = sum((U[i] / S)**2)
        for j in range(n):
            Q[i,j] = ((U[i,j]/S[j])**2)/Sum
    
    ListOfIndex = np.abs(np.max(S)/np.array(S))

    return Q, ListOfIndex

In [826]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

In [827]:
Models_list = [svm.SVC(kernel='rbf'), 
               RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0), 
               LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'),
                AdaBoostClassifier()]


Задача 1

In [297]:
def convert_bad_data(Data):
    Data_list = []
    for string in Data:
        Data_list.append(np.array([float(i) for i in str(string[0]).split()]))
    
    return np.array(Data_list)

In [1108]:
X_train = convert_bad_data(pd.read_csv('./data/task1_1_learn_X.csv').as_matrix())
y_train = np.int64(np.reshape(pd.read_csv('./data/task1_1_learn_y.csv').as_matrix(), -1))

X_test = convert_bad_data(pd.read_csv('./data/task1_1_test_X.csv').as_matrix())

In [1109]:
X = scale(np.vstack([X_train, X_test]))

In [1110]:
X_train = X[0:999]
X_test = X[999:1999]
print(X_train.shape)
print(X_test.shape)

(999, 20)
(999, 20)


In [1111]:
M = np.array([np.mean(X_train[np.where(y_train == 0)[0]].T, axis=1), np.mean(X_train[np.where(y_train == 1)[0]].T, axis=1)])
D = np.array([np.std(X_train[np.where(y_train == 0)[0]].T, axis=1), np.std(X_train[np.where(y_train == 1)[0]].T, axis=1)])


In [1112]:
print(M)

[[-0.12077668 -0.04348967 -0.1365637  -0.05275828 -0.13301657 -0.02688166
  -0.09471607 -0.05376665 -0.0798722  -0.06409888 -0.11279741 -0.10844195
  -0.10559884 -0.0029772  -0.09335749 -0.10795188 -0.06794247 -0.03708895
   0.01565226 -0.00281591]
 [ 0.07434899  0.05343222  0.16443539  0.08234838  0.11576175  0.08180707
   0.08586435  0.10301123  0.05550979  0.0747421   0.10576322  0.10890551
   0.12144663  0.10894485  0.06999953  0.08176019  0.08421166  0.0791003
   0.07569396  0.04164802]]


In [1123]:
count_of_clasters = 2
count_of_features = 20
mu, cov = EM_algorithm(np.vstack([X_train, X_test]), y_train, mu= mu, count_of_clasters=count_of_clasters, Num = 100)

# np.reshape(np.random.rand(count_of_clasters*count_of_features)- 0.5, [count_of_clasters, count_of_features])

In [1124]:
print(mu)

[[-0.12077668 -0.04348967 -0.1365637  -0.05275828 -0.13301657 -0.02688166
  -0.09471607 -0.05376665 -0.0798722  -0.06409888 -0.11279741 -0.10844195
  -0.10559884 -0.0029772  -0.09335749 -0.10795188 -0.06794247 -0.03708895
   0.01565226 -0.00281591]
 [ 0.07434899  0.05343222  0.16443539  0.08234838  0.11576175  0.08180707
   0.08586435  0.10301123  0.05550979  0.0747421   0.10576322  0.10890551
   0.12144663  0.10894485  0.06999953  0.08176019  0.08421166  0.0791003
   0.07569396  0.04164802]]


In [1130]:
B = np.array([_p_norm(X_train, mu[0], cov[0]), _p_norm(X_train, mu[1], cov[1])]).T
clustering = np.argmax(B, axis=1)
Data = []
Answ = []
for i in range(2):
    Data.append(np.array(X_train[np.where(clustering==i)]))
    Answ.append(np.array(y_train[np.where(clustering==i)]))
Data= np.array(Data)
Answ= np.array(Answ)

print(np.sum(Answ[0]), Answ[0].shape)
print(np.sum(Answ[1]), Answ[1].shape)

125 (510,)
357 (489,)


In [1131]:
answ3 = clustering

In [1126]:
B = np.array([_p_norm(np.vstack([X_train, X_test]), mu[0], cov[0]), _p_norm(np.vstack([X_train, X_test]), mu[1], cov[1])]).T
clustering = np.argmax(B, axis=1)
Data = []
Answ = []
for i in range(2):
    Data.append(np.array(np.vstack([X_train, X_test])[np.where(clustering==i)]))
    Answ.append(np.array(y_train[np.where(clustering[0:999]==i)]))
Data= np.array(Data)
Answ= np.array(Answ)

print(np.sum(Answ[0]), Data[0].shape)
print(np.sum(Answ[1]), Data[1].shape)

125 (1013, 20)
357 (985, 20)


In [1128]:
# Answer_test = []
# Answer_train = []
# Train_X, Test_X, Train_y, Test_y = train_test_split(X_train, y_train, test_size = 0.3)
# for model in Models_list:
#     model.fit(Train_X, Train_y)
#     answ = model.predict(Test_X)
#     Answer_test.append(get_err(Test_y, answ))
#     answ = model.predict(Train_X)
#     Answer_train.append(get_err(Train_y, answ))

# for i in range(len(Models_list)):
#     print_err_from_list(Answer_test[i])
#     print_err_from_list(Answer_train[i])
#     print('\n\n')

In [1132]:
A = np.array([(1-np.sum(y_train)/np.size(y_train))*np.prod(_p_norm_1d(X_train, M[0], D[0]**2), axis = 1), np.sum(y_train)/np.size(y_train)*np.prod(_p_norm_1d(X_train, M[1], D[1]**2), axis = 1)]).T
answ1 = np.argmax(A, axis=1)

B = np.array([_p_norm(X_train, mu[0], cov[0]), _p_norm(X_train, mu[1], cov[1])]).T
answ2 = np.argmax(B, axis=1)

print(answ1.shape)

print('Naive Bayes')
print_err(y_train, answ1)
print('\nEM')
print_err(y_train, answ2)
print('\n')
print_err(y_train, answ3)

(999,)
Naive Bayes
AUC 0.687833575447
NUM 310
ASY1: -2314
ASY2: -245
ACC: 0.68968968969

EM
AUC 0.742672375739
NUM 257
ASY1: -2405
ASY2: -353
ACC: 0.742742742743


AUC 0.742672375739
NUM 257
ASY1: -2405
ASY2: -353
ACC: 0.742742742743


In [807]:
print(M)

[[ 0.01483568 -0.02751732  0.019489    0.03495017 -0.74414272 -0.03755091
   0.04582135 -0.06099641 -0.03058157 -0.04987699 -0.00565696 -0.00763899
  -0.05878477 -0.00237989 -0.03561709 -0.00845051 -0.07784587  0.05099672
   0.06282502  0.09445809]
 [ 0.05151931  0.03543759 -0.02474086 -0.031153    0.81535173  0.05851961
  -0.03303321  0.04713858  0.01117343  0.00935614  0.05387784  0.03062816
   0.04166149  0.00769568 -0.01964896 -0.05315413  0.05947888 -0.00786346
  -0.07580418 -0.04740862]]


In [1070]:
# ax = plt.subplot() 
# sns.distplot(X_train[np.where(y_train == 0)[0]].T[2], kde=False, ax=ax, color = 'r')
# sns.distplot(X_train[np.where(y_train == 1)[0]].T[2], kde=False, ax=ax, color = 'b')


In [1069]:
# sns.pairplot(pd.DataFrame(np.hstack([X_train, y_train.reshape(-1,1)])))


Задача 2

In [880]:
X_train = convert_bad_data(pd.read_csv('./data/task1_2_learn_X.csv').as_matrix())
y_train = np.int64(np.reshape(pd.read_csv('./data/task1_2_learn_y.csv').as_matrix(), -1))

X_test = convert_bad_data(pd.read_csv('./data/task1_2_test_X.csv').as_matrix())

In [881]:
print(X_train.shape)

(999, 20)


In [882]:
M0 = []
D0 = []
M1 = []
D1 = []
for i in range(X_train.shape[1]):
    a = X_train[np.where(y_train == 0)[0]].T[i]
    b = X_train[np.where(y_train == 1)[0]].T[i]
    M0.append(np.mean(a))
    D0.append(np.std(a))
    M1.append(np.mean(b))
    D1.append(np.std(b))

M0 = np.array(M0)
M1 = np.array(M1)
D0 = np.array(D0)
D1 = np.array(D1)

M = np.array([M0, M1])
D = np.array([D0, D1])

In [886]:
print(M)

[[-0.03325893  0.02172398 -0.01058561 -0.04456297 -0.02852222 -0.03301072
  -0.0577591   0.00652275  0.01642043 -0.02551915  0.01294815  0.0271235
  -0.01720146  0.02224619  0.00281133  0.05723869 -0.0400518  -0.04018763
  -0.02258193  0.03659773]
 [ 0.06375896  0.01775647 -0.00643263  0.0841747   0.02382128  0.0018139
  -0.02501079  0.03620237 -0.16475624 -0.00118873 -0.00259019  0.05867702
  -0.02456966  0.04507624 -0.07664191 -0.03733185  0.03542011  0.0188795
   0.00621867  0.03643516]]


In [1134]:
count_of_clasters = 2
count_of_features = 20
mu, cov = EM_algorithm(np.vstack([X_train, X_test]), mu= mu, count_of_clasters=count_of_clasters, Num = 100)

# np.reshape(np.random.rand(count_of_clasters*count_of_features)- 0.5, [count_of_clasters, count_of_features])

TypeError: EM_algorithm() missing 1 required positional argument: 'y_train'

In [887]:
print(mu)

[[ 0.03383438  0.00782043  0.05279184 -0.03574432  0.0789249  -0.08390855
   0.05825984 -0.01623239 -0.10376422  0.06167183  0.00722287 -0.03024253
  -0.1374993   0.06037646  0.02038495  0.00123184  0.03451182  0.01445155
  -0.00467607  0.03790001]
 [-0.0275496   0.06780671 -0.00796216  0.06738307 -0.04408453  0.08710299
  -0.11863114  0.01619311  0.00260665 -0.07274448 -0.01850461  0.11653302
   0.13855958 -0.03794879 -0.09546509  0.03902559 -0.07832263 -0.04695737
  -0.02753907  0.00086828]]


In [885]:
B = np.array([_p_norm(X_train, mu[0], cov[0]), _p_norm(X_train, mu[1], cov[1])]).T
clustering = np.argmax(B, axis=1)
Data = []
Answ = []
for i in range(4):
    Data.append(np.array(X_train[np.where(clustering==i)]))
    Answ.append(np.array(y_train[np.where(clustering==i)]))
Data= np.array(Data)
Answ= np.array(Answ)

print(np.sum(Answ[0]))
print(np.sum(Answ[1]))

278
212


In [893]:
A = np.array([(1-np.sum(y_train)/np.size(y_train))*np.prod(_p_norm_1d(X_train, M[0], D[0]**2), axis = 1), np.sum(y_train)/np.size(y_train)*np.prod(_p_norm_1d(X_train, M[1], D[1]**2), axis = 1)]).T
answ1 = np.argmax(A, axis=1)

B = np.array([_p_norm(X_train, mu[0], cov[0]), _p_norm(X_train, mu[1], cov[1])]).T
answ2 = np.argmax(B, axis=1)

print(answ1.shape)

print('Naive Bayes')
print_err(y_train, answ1)
print('\nEM')
print_err(y_train, answ2)
print('\n')
print_err(y_train, y_train)

(999,)
Naive Bayes
AUC 0.592007136843
NUM 407
ASY1: -1035
ASY2: 7
ACC: 0.592592592593

EM
AUC 0.470746962832
NUM 528
ASY1: 188
ASY2: 307
ACC: 0.471471471471


AUC 1.0
NUM 0
ASY1: -5090
ASY2: -999
ACC: 1.0


In [373]:
print(M)

[[-0.03325893  0.02172398 -0.01058561 -0.04456297 -0.02852222 -0.03301072
  -0.0577591   0.00652275  0.01642043 -0.02551915  0.01294815  0.0271235
  -0.01720146  0.02224619  0.00281133  0.05723869 -0.0400518  -0.04018763
  -0.02258193  0.03659773]
 [ 0.06375896  0.01775647 -0.00643263  0.0841747   0.02382128  0.0018139
  -0.02501079  0.03620237 -0.16475624 -0.00118873 -0.00259019  0.05867702
  -0.02456966  0.04507624 -0.07664191 -0.03733185  0.03542011  0.0188795
   0.00621867  0.03643516]]


Задача 4

In [1071]:
X_train = convert_bad_data(pd.read_csv('./data/task1_4_learn_X.csv').as_matrix())
y_train = np.int64(np.reshape(pd.read_csv('./data/task1_4_learn_y.csv').as_matrix(), -1))

X_test = convert_bad_data(pd.read_csv('./data/task1_4_test_X.csv').as_matrix())

In [1072]:
print(X_train.shape)

(999, 200)


In [1137]:
print(mu)

[[-0.12077668 -0.04348967 -0.1365637  -0.05275828 -0.13301657 -0.02688166
  -0.09471607 -0.05376665 -0.0798722  -0.06409888 -0.11279741 -0.10844195
  -0.10559884 -0.0029772  -0.09335749 -0.10795188 -0.06794247 -0.03708895
   0.01565226 -0.00281591]
 [ 0.07434899  0.05343222  0.16443539  0.08234838  0.11576175  0.08180707
   0.08586435  0.10301123  0.05550979  0.0747421   0.10576322  0.10890551
   0.12144663  0.10894485  0.06999953  0.08176019  0.08421166  0.0791003
   0.07569396  0.04164802]]


In [1147]:
multivariate_normal.pdf(X_train, mean=mu[0], cov=cov[0])

ValueError: Array 'mean' must be a vector of length 40.

In [1141]:
_p_norm(X_train, mu[0], cov[0])

array([  1.30302721e-11,   1.42266836e-11,   5.12667637e-13,
         4.56175032e-13,   8.72382102e-11,   4.77048092e-13,
         1.01036204e-13,   4.30828313e-12,   3.18662774e-12,
         3.54292514e-12,   1.07337986e-11,   3.12649403e-14,
         5.86722587e-14,   1.04993710e-12,   1.99582179e-14,
         1.07819512e-11,   1.49191588e-14,   1.39258628e-11,
         1.22838558e-12,   1.70128142e-13,   1.74998198e-12,
         1.43257872e-12,   3.39611086e-11,   8.86864106e-12,
         1.50334751e-13,   1.93217025e-13,   5.98363527e-13,
         9.24984395e-15,   1.09924583e-11,   3.47451864e-13,
         6.47812639e-12,   4.58804775e-14,   5.10256784e-13,
         2.04910408e-17,   1.17038721e-11,   8.98218964e-14,
         2.17677481e-12,   4.33296944e-13,   8.44105754e-14,
         1.38405821e-14,   1.28786973e-16,   4.98135716e-14,
         2.71686668e-12,   7.22848375e-13,   1.97859428e-15,
         1.08307328e-14,   2.59769129e-12,   1.27475871e-12,
         1.40193265e-10,